# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [4]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [5]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [7]:
spaceship.dropna(how = 'any', inplace = True)

# Step 1: Extract the first letter (before "/")
spaceship['Cabin'] = spaceship['Cabin'].str[0]

spaceship.drop(columns = ['PassengerId', 'Name'], inplace = True)

features = spaceship.drop(columns = ['Transported'])
target = spaceship['Transported']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

# Convert non-numerical columns to dummies
df_dummies = pd.get_dummies(spaceship, drop_first=True)
df_dummies

features = df_dummies.drop(columns = ['Transported'])
target = df_dummies['Transported']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

In [8]:
X_train_reduced = X_train.drop(columns = ["Destination_PSO J318.5-22", "Cabin_T"])
X_test_reduced = X_test.drop(columns = ["Destination_PSO J318.5-22", "Cabin_T"])

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

# Initialize the Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100,        # Number of boosting stages (trees)
                                    learning_rate=0.1,       # Step size (smaller = better accuracy but slower training)
                                    max_depth=5,             # Maximum depth of each tree
                                    random_state=42)         # For reproducibility

# Fit the model on training data
gb_clf.fit(X_train_reduced, y_train)

# Make predictions on the test set
pred = gb_clf.predict(X_test_reduced)

# Evaluate the model using classification metrics
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred))
print("Recall:", recall_score(y_test, pred))

Accuracy: 0.8161875945537065
Precision: 0.8051575931232091
Recall: 0.8400597907324364


- Evaluate your model

we get an Accuracy of: 0.8161875945537065

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],  # Number of boosting stages
    'learning_rate': [0.01, 0.1],  # Step size
    'max_depth': [3, 5],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Min samples to split a node
    'min_samples_leaf': [1, 3, 5]  # Min samples required in a leaf node
}

# Initialize the Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(random_state=42)

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=gb_clf, 
                           param_grid=param_grid, 
                           scoring='accuracy',  # You can also use 'recall' if that's your priority
                           cv=3,  # 5-fold cross-validation
                           n_jobs=-1,  # Use all available processors
                           verbose=1)

# Fit GridSearchCV on training data
grid_search.fit(X_train_reduced, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model using the best parameters
best_gb_clf = GradientBoostingClassifier(**best_params, random_state=42)
best_gb_clf.fit(X_train_reduced, y_train)

# Make predictions on the test set
pred = best_gb_clf.predict(X_test_reduced)

# Evaluate the optimized model
print("Tuned Accuracy:", accuracy_score(y_test, pred))
print("Tuned Precision:", precision_score(y_test, pred))
print("Tuned Recall:", recall_score(y_test, pred))

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}
Tuned Accuracy: 0.8071104387291982
Tuned Precision: 0.7827868852459017
Tuned Recall: 0.8565022421524664


- Evaluate your model

we get a reduced Accuracy of: 0.8071104387291982